### Contracts for an AND logic gate composed of three subsystems using gear

Import the libraries:

In [1]:
import gear.iocontract as iocontract
import gear.polyhedralterm as polyhedralterm
from gear.gear import getVarset

Write contracts for subsystem 1 and subsystem 2:

In [2]:

contract1 = {
        "_comment1": "contract for subsystem 1, the assumption is that if input (u_1)"
        "is greater than u_1_min = 1, then it guarantees that the output (x_1) is greater than x_1_min = 1.5",
        "InputVars":[
            "u_1"
        ],
        "OutputVars":[
            "x_1"
        ],
        "assumptions":
        [
            {"coefficients":{"u_1":-1},
            "constant":-1}
        ],
        "guarantees":
        [
            {"coefficients":{"x_1":-1},
            "constant":-1.5}
        ]
    }
contract2 = {
        "_comment2": "contract for subsystem 2, the assumption is that if input (u_2)"
        "is greater than u_2_min = 1 then it guarantees that the output (x_2) is greater than x_2_min = 0.3",
        "InputVars":[
            "u_2"
        ],
        "OutputVars":[
            "x_2"
        ],
        "assumptions":
        [
            {"coefficients":{"u_2":-1},
            "constant":-1}
        ],
        "guarantees":
        [
            {"coefficients":{"x_2":-1},
            "constant":-0.3}
        ]
    }

Use `gear` to compute the composition of contract1 and contract2

In [3]:
contracts_json = [contract1, contract2]
contracts = []
for c in contracts_json:
    reqs = []
    for key in ['assumptions', 'guarantees']:
        reqs.append([polyhedralterm.PolyhedralTerm(term['coefficients'], term['constant']) for term in c[key]])
    cont = iocontract.IoContract(inputVars=getVarset(c['InputVars']), outputVars=getVarset(c['OutputVars']),
        assumptions=polyhedralterm.PolyhedralTermSet(set(reqs[0])), guarantees=polyhedralterm.PolyhedralTermSet(set(reqs[1])))
    contracts.append(cont)
print("Contract1:\n" + str(contracts[0]))
print("Contract2:\n" + str(contracts[1]))


Contract1:
InVars: {<Var u_1>}
OutVars:{<Var x_1>}
A: -1*u_1 <= -1
G: -1*x_1 <= -1.5
Contract2:
InVars: {<Var u_2>}
OutVars:{<Var x_2>}
A: -1*u_2 <= -1
G: -1*x_2 <= -0.30000000000000004


## Composition

In [4]:
contract_comp = contracts[0].compose(contracts[1])

In [5]:
print(contract_comp)

InVars: {<Var u_2>, <Var u_1>}
OutVars:{<Var x_1>, <Var x_2>}
A: -1*u_1 <= -1, -1*u_2 <= -1
G: -1*x_2 <= -0.30000000000000004, -1*x_1 <= -1.5


Post-process `contract_comp` (the composition of contract1 and contract2) (if needed as json)

In [6]:
contract12 = {}
contract12['InputVars']  = [str(var) for var in contract_comp.inputvars]
contract12['OutputVars'] = [str(var) for var in contract_comp.outputvars]
contract12['assumptions'] = [{'constant':str(term.constant), 'coefficients':{str(k):str(v) for k,v in term.variables.items()}} for term in contract_comp.a.terms]
contract12['guarantees'] = [{'constant':str(term.constant), 'coefficients':{str(k):str(v) for k,v in term.variables.items()}} for term in contract_comp.g.terms]

In [7]:
contract12

{'InputVars': ['u_2', 'u_1'],
 'OutputVars': ['x_1', 'x_2'],
 'assumptions': [{'constant': '-1', 'coefficients': {'u_1': '-1'}},
  {'constant': '-1', 'coefficients': {'u_2': '-1'}}],
 'guarantees': [{'constant': '-0.30000000000000004',
   'coefficients': {'x_2': '-1'}},
  {'constant': '-1.5', 'coefficients': {'x_1': '-1'}}]}

Write contract for third subsystem

In [8]:
contract3 = {
        "_comment3": "contract for subsystem 3, the assumption is that if input (x_1 and x_2) are greater than x_1_min = 1.5 and x_2_min = 0.3 then it guarantees that the output (y) is greater than y_eps = 1.5",
        "InputVars":[
            "x_1", "x_2"
        ],
        "OutputVars":[
            "y"
        ],
        "assumptions":
        [
            {"coefficients":{"x_1":-1},
            "constant":-1.5},
            {"coefficients":{"x_2":-1},
            "constant":-0.3}
        ],
        "guarantees":
        [
            {"coefficients":{"y":-1},
            "constant":-1.5}
        ]
    }

In [9]:
c = contract3
contracts = []
reqs = []
for key in ['assumptions', 'guarantees']:
    reqs.append([polyhedralterm.PolyhedralTerm(term['coefficients'], term['constant']) for term in c[key]])
cont = iocontract.IoContract(inputVars=getVarset(c['InputVars']), outputVars=getVarset(c['OutputVars']),
    assumptions=polyhedralterm.PolyhedralTermSet(set(reqs[0])), guarantees=polyhedralterm.PolyhedralTermSet(set(reqs[1])))
contracts.append(cont)
print("Contract3:\n" + str(contracts[0]))


Contract3:
InVars: {<Var x_1>, <Var x_2>}
OutVars:{<Var y>}
A: -1*x_2 <= -0.3, -1*x_1 <= -1.5
G: -1*y <= -1.5


Compose contract12 with contract3:

In [10]:
and_gate_contract = contract_comp.compose(contracts[0])

In [11]:
print(and_gate_contract)

InVars: {<Var u_2>, <Var u_1>}
OutVars:{<Var y>}
A: -1*u_1 <= -1, -1*u_2 <= -1
G: -1*y <= -1.5


## Quotient

Assume that we have `contract_comp` (the composition of `contract1` and `contract2`) and the `and_gate_contract` available. Synthesize the contract of subsystem 3 using quotient.

In [28]:
synthesized_contract = and_gate_contract.quotient(contract_comp)

In [29]:
print(synthesized_contract)

InVars: {<Var x_1>, <Var x_2>}
OutVars:{<Var y>}
A: 
G: -1*y <= -1.5


Clearly, the assumptions were not computed correctly for the quotient in this example as they are empty. I tried to debug this and found out that the call to `termset.simplify` in `deduce_with_context` (Line 518) makes the set of assumption empty. Maybe this is happening because, again, the terms are not helping each other in this example. 